In [1]:
from utils import get_data_loader
from datasets import get_target, get_source
import params

# load dataset
src_data_loader, src_ds = get_data_loader(params.src_dataset)
tgt_data_loader, tgt_ds = get_data_loader(params.tgt_dataset)

In [3]:
from models import Discriminator, LeNetClassifier, LeNetEncoder, ResnetEncoder

src_encoder = ResnetEncoder(pretrained=True)
tgt_encoder = ResnetEncoder()
critic = Discriminator(input_dims=params.d_input_dims,
                        hidden_dims=params.d_hidden_dims,
                    output_dims=params.d_output_dims)

In [8]:
import torch

criterion = torch.nn.CrossEntropyLoss()

# Extract the image tensor from the dataset tuples
images_src = src_ds[0][0]  # src_ds[0] is a tuple (image, label)
images_tgt = tgt_ds[0][0]  # tgt_ds[0] is a tuple (image, label)

images_src = images_src.unsqueeze(0)  # Add batch dimension
images_tgt = images_tgt.unsqueeze(0)  # Add batch dimension

# extract and concat features
feat_src = src_encoder(images_src) # Shape: [B, 500]
feat_tgt = tgt_encoder(images_tgt) # Shape: [B, 500]
print("Feature source", feat_src.size())
print("Feature target", feat_tgt.size())
feat_concat = torch.cat((feat_src, feat_tgt), 0) # Shape: [2*B, 500]

print("Feature concat", feat_concat.size())
# predict on discriminator
pred_concat = critic(feat_concat.detach())
print("Predict concat", pred_concat)

# prepare real and fake label
label_src = torch.ones(feat_src.size(0)).long()
print("Label source", label_src)
label_tgt = torch.zeros(feat_tgt.size(0)).long()
print("Label target", label_tgt)
label_concat = torch.cat((label_src, label_tgt), 0)
print("Label concat", label_concat)

# compute loss for critic
loss_critic = criterion(pred_concat, label_concat)

Feature source torch.Size([1, 500])
Feature target torch.Size([1, 500])
Feature concat torch.Size([2, 500])
Predict concat tensor([[-0.6953, -0.6910],
        [-0.6950, -0.6913]], grad_fn=<LogSoftmaxBackward0>)
Label source tensor([1])
Label target tensor([0])
Label concat tensor([1, 0])


In [14]:
# visualize to compare feat_src and feat_tgt
import matplotlib.pyplot as plt
import numpy as np
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

def visualize_features(features, labels, title):
    # Perform PCA
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(features)

    # Create a scatter plot
    plt.figure(figsize=(8, 6))
    scatter = plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=labels, cmap='viridis', alpha=0.5)
    plt.title(title)
    plt.colorbar(scatter)
    plt.xlabel('PCA Component 1')
    plt.ylabel('PCA Component 2')
    plt.show()
# Visualize the features
visualize_features(feat_src.detach().numpy(), label_src.numpy(), 'Source Features')
visualize_features(feat_tgt.detach().numpy(), label_tgt.numpy(), 'Target Features')

ValueError: n_components=2 must be between 0 and min(n_samples, n_features)=1 with svd_solver='full'